In [88]:
import pandas as pd

#import data into dataframe
dataSource = 'https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv'
df = pd.read_csv(dataSource, index_col=0)



#drop all null columns
df = df.dropna(how='all', axis=1)

#drop country_region_code because all are US anyways
df=df.drop(columns='country_region_code')

#drop rows with too many Null values for symptoms. If > 50% null, drop
#because 121 symtpoms, > 60 Nulls, row gets dropped
df=df.dropna(axis = 0, thresh=60)

#Drop this column because it is the same as open_covid_region_code
df=df.drop(columns='sub_region_1_code')


df['date'] = pd.to_datetime(df['date'])

#create multi-index for proper merging
df = df.set_index([df.index,'date'])



#code to compress and export to csv

df.to_csv('out.csv')



(234, 123)


In [89]:
import numpy as np
from datetime import datetime

url = 'https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv'
df2 = pd.read_csv(url,index_col=0,parse_dates=[0])



#replace all 0s with Nulls to drop

df2.replace(0, np.nan, inplace=True)



#only interested in hospitalized_new data entries
columns_needed= ['region_name', 'date', 'hospitalized_new']
df2 = df2[columns_needed]


#if no hospitalized_new data, row not needed
df2=df2.dropna(subset = ['hospitalized_new'])








#convert daily data to weekly data in order for future merging
#data starts on Mondays for symptoms dataset
df2['date'] = pd.to_datetime(df2['date'])
df2=df2.set_index('date').groupby([df2.index, 'region_name']).resample('W', label='left', loffset=pd.DateOffset(days=1)).sum().fillna(0).reset_index().set_index('open_covid_region_code')






#create multiindex for merging later
df2 = df2.set_index([df2.index,'date'])


#export to csv
df2.to_csv('hospital.csv')







MultiIndex([(  'CHE', '2020-02-24'),
            (  'CHE', '2020-03-02'),
            (  'CHE', '2020-03-09'),
            (  'CHE', '2020-03-16'),
            (  'CHE', '2020-03-23'),
            (  'CHE', '2020-03-30'),
            (  'CHE', '2020-04-06'),
            (  'CHE', '2020-04-13'),
            (  'CHE', '2020-04-20'),
            (  'CHE', '2020-04-27'),
            ...
            ('US-WY', '2020-08-03'),
            ('US-WY', '2020-08-10'),
            ('US-WY', '2020-08-17'),
            ('US-WY', '2020-08-24'),
            ('US-WY', '2020-08-31'),
            ('US-WY', '2020-09-07'),
            ('US-WY', '2020-09-14'),
            ('US-WY', '2020-09-21'),
            ('US-WY', '2020-09-28'),
            ('US-WY', '2020-10-05')],
           names=['open_covid_region_code', 'date'], length=4483)
(4483, 2)


In [137]:


from sklearn import preprocessing

# Set charts to view inline
%matplotlib inline

#inner join
finalDf = pd.merge(df, df2, left_index=True, right_index=True)


#left join, regions without hospitalization data still show up
finalDfLeft = left_df = pd.merge(df, df2, left_index=True, right_index=True, how='left')


finalDf.to_csv('finalInner.csv')
finalDfLeft.to_csv('finalLeft.csv')


#normalizing finalDfLeft
#normalizing by taking each region,
#using Min-Max preprocessing for the feature data


#columns that are not floating point values get droppped for each dataframe
droppingCols = ['country_region','sub_region_1','region_name','hospitalized_new']
country_region = finalDfLeft['country_region']
sub_region_1 = finalDfLeft['sub_region_1']
hospitalized_new = finalDfLeft['hospitalized_new']
date = finalDfLeft.index.get_level_values(1)

#Alaska
alaskaDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'Alaska']
alaskaDf.drop(droppingCols, axis = 1, inplace = True)


x = alaskaDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
alaskaDfNormal = pd.DataFrame(x_scaled, columns = alaskaDf.columns)

#Montana
montanaDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'Montana']
montanaDf.drop(droppingCols, axis = 1, inplace = True)

x = montanaDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
montanaDfNormal = pd.DataFrame(x_scaled, columns = montanaDf.columns)
    

#North Dakota
northdakotaDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'North Dakota']
northdakotaDf.drop(droppingCols, axis = 1, inplace = True)

x = northdakotaDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
northdakotaNormal = pd.DataFrame(x_scaled, columns = northdakotaDf.columns)

#South Dakota
southdakotaDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'South Dakota']
southdakotaDf.drop(droppingCols, axis = 1, inplace = True)

x = southdakotaDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
southdakotaNormal = pd.DataFrame(x_scaled, columns = southdakotaDf.columns)

#Vermont
vermontDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'Vermont']
vermontDf.drop(droppingCols, axis = 1, inplace = True)

x = vermontDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
vermontDfNormal = pd.DataFrame(x_scaled, columns = vermontDf.columns)

#Wyoming
wyomingDf = finalDfLeft[finalDfLeft['sub_region_1'] == 'Wyoming']
wyomingDf.drop(droppingCols, axis = 1, inplace = True)

x = wyomingDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
wyomingDfNormal = pd.DataFrame(x_scaled, columns = wyomingDf.columns)

#concatonate all normalized dataframes
frames = [alaskaDfNormal,montanaDfNormal, northdakotaNormal, southdakotaNormal,vermontDfNormal,wyomingDfNormal]

finalDataframeNormalized = pd.concat(frames)

#reinsert columns that could not be normalized bc they were of type str
finalDataframeNormalized.insert(0,'country_region',country_region.values)
finalDataframeNormalized.insert(1,'sub_region_1',sub_region_1.values)
finalDataframeNormalized.insert(2,'hospitalized_new',hospitalized_new.values)
finalDataframeNormalized.insert(3,'date',date.values)
print(finalDataframeNormalized.index)




finalDataframeNormalized.to_csv("finalNormalTotal.csv")


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
           dtype='int64', length=234)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:373: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:374: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages